In [1]:
import glob
import functools
import urllib2

import pandas as pd
import json
import gspread
import pandas as pd
import os
from IPython.core.display import HTML
from oauth2client.client import SignedJwtAssertionCredentials
import numpy as np
import shutil
from collections import Counter
import re
import time
pd.set_option("display.max_colwidth", 1000)

In [2]:
# index: row from dataframe that represents the manifest
# adapter: R1 or R2 for the reads
# returns: location on TSCC
def format_file(index, adapter):
    index_1 = illumina_adapters.ix[index.index_1].values[0]
    index_2 = illumina_adapters.ix[index.index_2].values[0]
    if index.Lane == "": #Incase we are doing a rapid run and there isn't lane info
        sample_name = "{}_{}-{}_{}.fastq.gz".format(index.Hiseq_file_name, index_2, index_1, adapter)
        dir_name = "Sample_{}".format(index.Hiseq_file_name)

    #For Sebastians samples Sample_G3BP-S-input/G3BP-S-input_S1_L001_R1.fastq.gz
    elif index.Lane.startswith("S") and "L" in index.Lane:
        name = index.Hiseq_file_name
        sample_name = "{}_{}_{}.fastq.gz".format(name, index.Lane, adapter)
        dir_name = index.Hiseq_file_name
    
    elif index.Lane.startswith("S"):
        name = index.Hiseq_file_name.split("-")[0].replace("_", "-")
        sample_name = "{}_{}_{}.fastq.gz".format(name, index.Lane, adapter)
        dir_name = index.Hiseq_file_name
 
    #This needs to go above my other 4000 check because of name colisions
    elif index.is_4000:
        sample_name = "{}_{}_{}_001.fastq.gz".format(index.Hiseq_file_name, index.Lane, adapter)
        return os.path.join(index.file_location, sample_name)
    
    #this is a bit hacky, keep an eye on it
    elif "4000" in index.Hiseq_file_name:
        sample_name = index.Hiseq_file_name + ".fastq.gz"
        sample_name = sample_name.replace("R1", adapter)
        return os.path.join(index.file_location, sample_name)
      
    else:
        sample_name = "{}_{}-{}_{}_{}.fastq.gz".format(index.Hiseq_file_name, index_2, index_1, index.Lane, adapter)
        dir_name = "Sample_{}".format(index.Hiseq_file_name)
    return os.path.join(index.file_location, dir_name, sample_name)

In [3]:
try:
    illumina_adapters = """501 TATAGCCT 
    502 ATAGAGGC 
    503 CCTATCCT 
    504 GGCTCTGA 
    505 AGGCGAAG 
    506 TAATCTTA 
    507 CAGGACGT 
    508 GTACTGAC 
    502s ATA 
    503s CCT
    504s GGC
    505,506 eric_is_stupid
    505s AGG
    506s TAA
    507s CAG
    508s GTA
    701 ATTACTCG
    702 TCCGGAGA
    703 CGCTCATT
    704 GAGATTCC
    705 ATTCAGAA
    706 GAATTCGT""".split("\n")

    illumina_adapters = pd.DataFrame([item.strip().split() for item in illumina_adapters], columns=["label", "barcode"])
    illumina_adapters = illumina_adapters.set_index("label")

    json_key = json.load(open("/home/gpratt/ipython_notebook/public clip-588adbc137f3.json"))
    scope = ['https://spreadsheets.google.com/feeds']

    credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
    gc = gspread.authorize(credentials)

    sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")
    ws = sht1.worksheet("Sheet1")
    list_of_lists = ws.get_all_values()
    manifest = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
    manifest.is_encode = manifest.is_encode == "TRUE"
    manifest.is_4000 = manifest.is_4000 == "TRUE"

    manifest.method_Paper_flag = manifest.Method_Paper_flag == "TRUE" 
    manifest['exp_id'] = manifest.ENCODE_ID.apply(lambda x: x.split("_")[0])

    manifest['r1'] = manifest.apply(functools.partial(format_file, adapter="R1"), axis=1)
    manifest['r2'] = manifest.apply(functools.partial(format_file, adapter="R2"), axis=1)
except KeyError as e:
    print e

/home/bay001/anaconda2/envs/brian/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


('', u'occurred at index 2251')


In [4]:
print(manifest.shape) # number of samples
manifest.head()

(2276, 26)


,Hiseq_file_name,ENCODE_ID,RBP,inline_1,inline_2,index_1,index_2,Lane,file_location,unmerged_location,...,Method_Paper_flag,species,is_4000,,,,,,,exp_id
0,222_CLIP,222_01,HNRNPM,C01,D08fixed,503,701,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,...,TRUE,hg19,False,,,,,,,222
1,222_CLIP,222_02,HNRNPM,A01,B06,503,701,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,...,TRUE,hg19,False,,,,,,,222
2,222_INPUT,222_INPUT,HNRNPM,none,none,502,701,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,...,TRUE,hg19,False,,,,,,,222
3,223_CLIP,223_01,FKBP4,A01,B06,503,702,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,...,TRUE,hg19,False,,,,,,,223
4,223_CLIP,223_02,FKBP4,C01,D08fixed,503,702,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,...,TRUE,hg19,False,,,,,,,223


In [5]:
output_file = '/projects/ps-yeolab3/bay001/encode/20171120_eCLIP_manifest.txt'
manifest.to_csv(
    output_file, sep='\t'
)